In [3]:
# In[1]:
import time
t1 = time.time()

#importando as bibliotecas 
import torch
import torch_geometric
import numpy as np
import sys 
from torch_geometric.data import Data
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv 
import statistics as sts
import matplotlib.pyplot as plt

sys.path.insert(1, '/home/marina/InCi/')
from cbirdb import acbirdb,cbirdbDistMatrix,evaluator,unsuperalg,recknngraph

execucoes = 10

def DasProgrammeHausaufgabe(DistM1, DistM2, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs): 

    # máscaras 

    train_mask1 = [] #máscaras de treino - true = treino 
    val_mask1 = [] #valores das máscaras 
    test_mask1 = [] #máscaras de testes 
    y1 = [] # y = vetor de "gabarito" 
    
    train_mask2 = [] #máscaras de treino - true = treino 
    val_mask2 = [] #valores das máscaras 
    test_mask2 = [] #máscaras de testes 
    y2 = [] # y = vetor de "gabarito" 

    for i in range (pN):# para cada imagem i: 
        y1.append(i // classSize) # definindo o vetor de classes 
        d = i % classSize # d = resto do número da imagem dividido pela classe 
        if (d<trPerClass): # se d é menor que o número de imagens de treino por classe, então: 
            valueTr = True # treino = true 
            valueVal = False # teste = false 
        else: # d é maior ou igual ao número de imagens de treino por classe: 
            valueTr = False # treino = false 
            if (d<(valPerClass+trPerClass)): 
                valueVal = True # teste = true 
            else:
                valueVal = False #teste = false 
        valueTest = (not valueTr) and (not valueVal) 

        train_mask1.append (valueTr) # criando os vetores de máscaras 
        val_mask1.append(valueVal) #vetor de validação 
        test_mask1.append(valueTest)
        
    for i in range (pN):# para cada imagem i: 
        y2.append(i // classSize) # definindo o vetor de classes 
        d = i % classSize # d = resto do número da imagem dividido pela classe 
        if (d<trPerClass): # se d é menor que o número de imagens de treino por classe, então: 
            valueTr = True # treino = true 
            valueVal = False # teste = false 
        else: # d é maior ou igual ao número de imagens de treino por classe: 
            valueTr = False # treino = false 
            if (d<(valPerClass+trPerClass)): 
                valueVal = True # teste = true 
            else:
                valueVal = False #teste = false 
        valueTest = (not valueTr) and (not valueVal) 

        train_mask2.append (valueTr) # criando os vetores de máscaras 
        val_mask2.append(valueVal) #vetor de validação 
        test_mask2.append(valueTest)

    # A torch.Tensor is a multi-dimensional matrix containing elements of a single data type.
    train_mask1 = torch.tensor (train_mask1) #torch 
    val_mask1 = torch.tensor (val_mask1)
    test_mask1 = torch.tensor (test_mask1)
    y1 = torch.tensor (y1)
    
    train_mask2 = torch.tensor (train_mask2) #torch 
    val_mask2 = torch.tensor (val_mask2)
    test_mask2 = torch.tensor (test_mask2)
    y2 = torch.tensor (y2)

    
    #-------------------------------- Feature Matrix ------------------------------------- 
    #lendo a matriz de features 

    #print ('Reading feature matrix ...')
    feat_matrix_file = "/home/marina/Documents/H/matrix_sem_virgulas.txt"
    x1 = np.loadtxt(feat_matrix_file, delimiter=" ")
    #print(x.shape)
    x1 = torch.tensor(x1)
    #print (x)
    
    # Indexação das Bordas 

    #arquivo lista 
    pListFile = "list.txt"
    #arquivo dist matrix
    pDSimFile1 = DistM1 #"/home/marina/venv/oxford17flowers-caffe-fc7_matrix.txt"
    pDSimFile2 = DistM2
    
    #x2 = np.loadtxt("matrix_flowers_rlsim_bonita_corrigida.txt", delimiter=" ")
    x2 = np.loadtxt(DistM2, delimiter=" ")
    #print(x.shape)
    x2 = torch.tensor(x2)

    #print ('Reading DB ...')
    db1 = cbirdbDistMatrix(pListFile,pDSimFile1,pN)
    db2 = cbirdbDistMatrix(pListFile,pDSimFile2,pN)

    #print ('Making edge list ...')
    edge_index1 = []
    edge_index2 = []

    for qimg in db1.getList(): #para cada imagem da lista 
        # DB ORIGINAL - NO RESTO USAR O DB NOVO 
        #print (qimg)
        crl = db1.getRL(qimg) # mais próximos no ranked list 
        qindex = db1.getIndexOf(qimg) 
        for i in range(pK): # cria arestas entre as k posições mais próximas 
            cindex = db1.getIndexOf (crl[i])
            edge_index1.append([qindex,cindex])
            
    for qimg in db2.getList(): #para cada imagem da lista 
        # DB ORIGINAL - NO RESTO USAR O DB NOVO 
        #print (qimg)
        crl = db2.getRL(qimg) # mais próximos no ranked list 
        qindex = db2.getIndexOf(qimg) 
        for i in range(pK): # cria arestas entre as k posições mais próximas 
            cindex = db2.getIndexOf (crl[i])
            edge_index2.append([qindex,cindex])

    #passa para o torch 
    # A torch.Tensor is a multi-dimensional matrix containing elements of a single data type.
    edge_index1 = torch.tensor(edge_index1)
    edge_index2 = torch.tensor(edge_index2)
    edge_index2 = edge_index2.t().contiguous()

    #print (edge_index)
    
    #--------------------------------- Data Object  -----------------------------------------
    #data são as informações do grafo completo (eu acho) *_* 

    #print ('Loading data object...')
    data1 = Data(x1=x1.float(), edge_index1=edge_index1, y1=y1, test_mask1=test_mask1, train_mask1=train_mask1,  val_mask1=val_mask1  )
    data2 = Data(x2=x2.float(), edge_index2=edge_index2, y2=y2, test_mask2=test_mask2, train_mask2=train_mask2,  val_mask2=val_mask2  )


    #--------------------------------- Graph Neural Network Defiition  -----------------------------------------

    # rede neural: NÃO MEXA!!!!!!!!!!

    #print ('Defining GCN model...')

    class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = GCNConv(pNFeatures, pNNeurons) #dataset.num_node_features
            self.conv2 = GCNConv(pNNeurons, numberOfClasses) #dataset.num_classes

        def forward(self, data):
            x, edge_index = data.x, data.edge_index

            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
            x = self.conv2(x, edge_index)

            return F.log_softmax(x, dim=1)



    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = torch.device('cpu')
    model1 = Net().to(device)
    model2 = Net().to(device)
    #data = dataset[0].to(device)
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=pLR, weight_decay=5e-4)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=pLR, weight_decay=5e-4)

    #print ('Training...')

    model1.train() # treinando com a rede neural 
    model2.train() # treinando com a rede neural 
    
    for epoch in range(pNEpochs):	
        #print("Training epoch: ", epoch)
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        out1 = model1(data1)
        print("OUT 1: ", out1)
        out2 = model2(data2) 
        loss1 = F1.nll_loss(out1[data1.train_mask1], data1.y1[data1.train_mask1])
        loss2 = F2.nll_loss(out2[data2.train_mask2], data2.y2[data2.train_mask2])
        loss1.backward()
        optimizer1.step()
        loss2.backward()
        optimizer2.step()


    # aqui ele avalia o modelo, com base nas máscaras de teste e treino que definimos no começo     
    model1.eval()
    _, pred1 = model1(data1).max(dim=1)
    correct1 = float (pred[data1.test_mask1].eq(data1.y1[data1.test_mask1]).sum().item())
    acc1 = correct1 / pN #data.test_mask.sum().item() # esse acc é o mesmo da soccer? é um descritor? 

    model2.eval()
    _, pred2 = model2(data2).max(dim=1)
    correct2 = float (pred[data2.test_mask2].eq(data2.y2[data2.test_mask2]).sum().item())
    acc2 = correct2 / pN
    
    #print('Accuracy: {:.4f}'.format(acc)) 
    return(acc1, acc2)


# informações do dataset
pN = 1360 
classSize = 80
numberOfClasses = 17

pNFeatures = 4096 # número de features 

# parâmetros 
trPerClass = 15 #treinamento por classe 
valPerClass = 0*trPerClass #valores por classe 

pK = 35 # número de vizinhos mais próximos (serve para gerar o grafo)
pLR = 0.001 # learning rate: o quão rápido os pesos são atualizados (não mexer)
pNNeurons = 500 # quantidade de neurônios nas camadas intermediárias (serve para otimizar)
pNEpochs = 300 # quantidade de interações de treinamento a cada época (calcula e ajusta os pesos)
# época = rodada de ajustes de pesos

DistM1 = "matrix_sem_virgulas.txt" 
DistM2 = "matrix_flowers_rlsim_bonita_corrigida.txt" #"CPRR.txt"

Media_NM = 0
dp_original = []

for i in range(11): 
    print(i)
    NM1, NM2 = DasProgrammeHausaufgabe(DistM1, DistM2, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_original.append(NM)
    Media_NM = Media_NM + NM

PoD = 0 
dp_udlf = []
for i in range(11): 
    print(i)
    AP = DasProgrammeHausaufgabe(DistM1, DistM2, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_udlf.append(AP)
    PoD = PoD + AP


Media_NM = Media_NM/execucoes 
print("Nova Matriz sem UDLF: ", Media_NM, "\nDesvio Padrão: ", sts.stdev(dp_original))
PoD = PoD/execucoes
print("Nova Matriz com UDLF: ", PoD, "\nDesvio Padrão: ", sts.stdev(dp_udlf))


# In[ ]:
t2 = time.time()
print("Tempo de execução: ", (t2-t1)/60 )

0


TypeError: linear(): argument 'input' (position 1) must be Tensor, not NoneType

DistM = "LHRR.txt"
PoD = 0 
dp_udlf = []
for i in range(11): 
    print(i)
    AP = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_udlf.append(AP)
    PoD = PoD + AP

PoD = PoD/execucoes
print("Nova Matriz com UDLF: ", PoD, "\nDesvio Padrão: ", sts.stdev(dp_udlf))


# In[ ]:
t2 = time.time()
print("Tempo de execução: ", (t2-t1)/60 )

DistM = "CORGRAPH.txt"
PoD = 0 
dp_udlf = []
for i in range(11): 
    print(i)
    AP = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_udlf.append(AP)
    PoD = PoD + AP

PoD = PoD/execucoes
print("Nova Matriz com UDLF: ", PoD, "\nDesvio Padrão: ", sts.stdev(dp_udlf))


# In[ ]:
t2 = time.time()
print("Tempo de execução: ", (t2-t1)/60 )

DistM = "BFSTREE.txt"
PoD = 0 
dp_udlf = []
for i in range(11): 
    print(i)
    AP = DasProgrammeHausaufgabe(DistM, pN, classSize, numberOfClasses, pNFeatures, trPerClass, valPerClass, pK, pLR, pNNeurons, pNEpochs)
    dp_udlf.append(AP)
    PoD = PoD + AP

PoD = PoD/execucoes
print("Nova Matriz com UDLF: ", PoD, "\nDesvio Padrão: ", sts.stdev(dp_udlf))


# In[ ]:
t2 = time.time()
print("Tempo de execução: ", (t2-t1)/60 )